In [96]:
# load the packages needed to run the code
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import re
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
#from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

In [97]:
activity_list = ['Backpacking',
                 'Bird watching',
                 'Camping',
                 'Cross country skiing',
                 'Fishing',
                 'Hiking',
                 'Horseback riding',
                 'Mountain biking',
                 'Nature trips',
                 'Off-road driving',
                 'Paddle sports',
                 'Road biking',
                 'Rock climbing',
                 'Scenic driving',
                 'Skiing',
                 'Snowshoeing',
                 'Surfing',
                 'Running',
                 'Walking']

attraction_list = ['Beach',
                   'Cave',
                   'City walk',
                   'Forest',
                   'Historic site',
                   'Hot springs',
                   'Lake',
                   'Pub walk',
                   'Rails trails',
                   'River',
                   'Views',
                   'Waterfall',
                   'Wild flowers',
                   'Wildlife']

suitability_list = ['Dog friendly',
                    'Kid friendly',
                    'Partially paved',
                    'Paved',
                    'Stroller friendly',
                    'Wheelchair friendly']

In [98]:
# specify the path where you want to save the .csv file
#outpath = '/Users/spk/Documents/Code/Python/webscrape_proj/outputs/banff_national_park_trailforks.csv'
outDIR = '/Users/spk/Documents/GitHub/trail_scraping/outputs/'
if not os.path.exists(outDIR):
    os.mkdir(outDIR)
out_name = 'AllTrails_Kananaskis_CPLU.csv'
# specify the url that you want to scrape from
url = 'https://www.alltrails.com/parks/canada/alberta/kananaskis-country-public-land-use-zone'

In [99]:
options = webdriver.ChromeOptions()
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--no-sandbox') 
options.add_argument('--disable-dev-shm-usage')
options.add_argument('window-size=1920,1080')

driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.GOOGLE).install(),
                         options=options)
#driver.maximize_window()
#driver.execute_script("document.body.style.zoom='50%'")
timeout = 5

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - There is no [mac64] chromedriver for browser 89.0.4389 in cache
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/spk/.wdm/drivers/chromedriver/mac64/89.0.4389.23]


In [100]:
driver.get(url)
trail_table_elem = WebDriverWait(driver, timeout)\
    .until(ec.presence_of_element_located((By.CLASS_NAME, 'styles-module__results___24LBd')))

In [101]:
df_out = pd.DataFrame(columns=['Name', 'URL', 'Zone', 'Difficulty', 'Rating', 'Reviewers', 'Length_km'])
idx = 1
idx_bool = True
while idx_bool:
    xpath_tmp = '//*[@id="stick-bar-parent"]/div[2]/div[1]/div[2]/div[{}]'.format(str(idx))
    try:
        elem_tmp = trail_table_elem.find_elements(By.XPATH, xpath_tmp)
        if len(elem_tmp) == 1:
            df_out.loc[idx-1, 'Name'] = trail_table_elem.find_element(By.XPATH, 
                                          xpath_tmp + '/a').get_attribute('text')
            print(df_out.loc[idx-1, 'Name'])
            df_out.loc[idx-1, 'URL'] = trail_table_elem.find_element(By.XPATH, 
                                          xpath_tmp + '/a').get_attribute('href')
            df_out.loc[idx-1, 'Zone'] = trail_table_elem.find_element(By.XPATH, 
                                          xpath_tmp + '/div[2]/a').get_attribute('text')
            df_out.loc[idx-1, 'Difficulty'] = trail_table_elem.find_element(By.XPATH,
                                            xpath_tmp + '/div[2]/div[2]/span[1]').text
            df_out.loc[idx-1, 'Rating'] = re.sub(' Stars', '', trail_table_elem.find_element(By.XPATH,
                                    xpath_tmp + '/div[2]/div[2]/span[2]').get_attribute('aria-label'))
            df_out.loc[idx-1, 'Reviewers'] = int(trail_table_elem.find_element(By.XPATH,
                            xpath_tmp + '/div[2]/div[2]/span[3]').text.replace('(', '').replace(')', ''))
            length_label = trail_table_elem.find_element(By.XPATH,
                    xpath_tmp + '/div[2]/div[3]/span[1]').text
            length_label = re.sub('Length: ', '', length_label)
            if bool(re.search(' mi', length_label)):
                df_out.loc[idx-1, 'Length_km'] = round(
                    float(re.sub(' mi', '', length_label)) * 1.60934, 2)
            elif bool(re.search(' km', length_label)):
                df_out.loc[idx-1, 'Length_km'] = float(re.sub(' km', '', length_label))
            idx += 1
        elif len(elem_tmp) == 0:
            print("\nNo more elements found, trying next page...\n")
            next_page = driver.find_element(By.CLASS_NAME, "styles-module__button___1nuva ")
            if next_page is None:
                print("\nScraping finished - no more pages")
                idx_bool = False
            else:
                next_page.click()
        else:
            print('\nERROR: more than 1 element matches XPATH query: ' + xpath_tmp)
            idx_bool = False
    except NoSuchElementException:
        print("\nInitial scraping finished - no more trails or pages")
        idx_bool = False

Prairie Mountain
Jumpingpound Summit Trail
Moose Mountain Trail
Powderface Creek and Prairie Creek Trail Loop
Wasootch Peak via Kananaskis Peak Route
Prairie Mountain and Prairie Creek
Carnarvon Lake
Mesa Butte
Elbow Valley Trail to Riverview Trail Loop
Forgetmenot Mountain North Summit via Wildhorse Trail

No more elements found, trying next page...

Powderface East Ridge Trail
Jumpingpound Ridge Trail
Square and Mesa Butte via Mesa Grind Trail
High Noon Trail
Jumpingpound Summit and Ridge Trails
Moose Mountain Northeast Summit
Curley Sands Loop
Mount Bryant Lake
Horton Hill
Hunchback Hills to Lusk Ridge South

No more elements found, trying next page...

Sulphur Springs Trail
Mesa Bute via Mesa Traverse Trail Loop
Pine Ridge and Macabee Loop Trail
Sugar Momma, Sugar Daddy, and Snagmore Loop
Mustang Hills East Summit Route
Mount Bryant Loop
Jumpingpound Mountain
Tiara Peak
High Noon to Gem Loop
Cat Creek Hills via Kananaskis Trail

No more elements found, trying next page...

Zephyr C

In [102]:
for idx, url in enumerate(df_out['URL']):
    print(df_out.loc[idx, 'Name'])
    driver.get(url)
    elev_gain = WebDriverWait(driver, timeout)\
    .until(ec.presence_of_element_located(
        (By.XPATH,
         '//*[@id="main"]/div[2]/div[1]/article/section[2]/div/span[2]/span[2]'))).text
    df_out.loc[idx, 'Route type'] = driver.find_element(
        By.XPATH,
        '//*[@id="main"]/div[2]/div[1]/article/section[2]/div/span[3]/span[2]').text
    if bool(re.search(' ft', elev_gain)):
        df_out.loc[idx, 'Elevation_gain_m']  = round(
            float(re.sub(' ft', '', elev_gain).replace(',', '')) * 0.3048, 2)
    else:
        df_out.loc[idx, 'Elevation_gain_m']  = round(
            float(re.sub(' m', '', elev_gain).replace(',', '')) * 0.3048, 2)
    
    tag_cloud = driver.find_elements(By.XPATH, 
                                     '//*[@id="main"]/div[2]/div[1]/article/section[3]/*')
    activities = []
    attractions = []
    suitabilities = []
    for tag in tag_cloud:
        if tag.text in activity_list:
            activities.append(tag.text)
        if tag.text in attraction_list:
            attractions.append(tag.text)
        if tag.text in suitability_list:
            suitabilities.append(tag.text)
    df_out.loc[idx, 'Activities'] = ', '.join(activities)
    df_out.loc[idx, 'Attractions'] = ', '.join(attractions)
    df_out.loc[idx, 'Suitabilities'] = ', '.join(suitabilities)

Prairie Mountain
Jumpingpound Summit Trail
Moose Mountain Trail
Powderface Creek and Prairie Creek Trail Loop
Wasootch Peak via Kananaskis Peak Route
Prairie Mountain and Prairie Creek
Carnarvon Lake
Mesa Butte
Elbow Valley Trail to Riverview Trail Loop
Forgetmenot Mountain North Summit via Wildhorse Trail
Powderface East Ridge Trail
Jumpingpound Ridge Trail
Square and Mesa Butte via Mesa Grind Trail
High Noon Trail
Jumpingpound Summit and Ridge Trails
Moose Mountain Northeast Summit
Curley Sands Loop
Mount Bryant Lake
Horton Hill
Hunchback Hills to Lusk Ridge South
Sulphur Springs Trail
Mesa Bute via Mesa Traverse Trail Loop
Pine Ridge and Macabee Loop Trail
Sugar Momma, Sugar Daddy, and Snagmore Loop
Mustang Hills East Summit Route
Mount Bryant Loop
Jumpingpound Mountain
Tiara Peak
High Noon to Gem Loop
Cat Creek Hills via Kananaskis Trail
Zephyr Creek/Painted Canyon Trail
Allsmoke Mountain Trail
Lower Cataract Creek
Lake of the Horns
Death's Head Short Route
Belmore Browne Peak and 

In [95]:
df_out.to_csv(os.path.join(outDIR, out_name), sep=',', index=False)

In [93]:
outDIR

'/Users/spk/Documents/Code/Python/webscrape_proj/outputs/'

In [ ]:
//*[@id="stick-bar-parent"]/div[2]/div[1]/div[2]/div[1]
//*[@id="stick-bar-parent"]/div[2]/div[1]/div[2]/div[2]
//*[@id="stick-bar-parent"]/div[2]/div[1]/div[2]/div[3]